### Fase 1: La Plaza 
Te encuentras en la plaza de la ciudad, donde los habitantes desarrollan sus diversas profesiones: panaderos, herreros, guardias, escuderos...   
En el siguiente enlace https://cutt.ly/peVAXig  encontrarás un archivo que contiene un número por cada persona que se encuentra en la plaza. Suma los números que correspondan a los guardias y podrás pasar a la siguiente fase.


In [1]:
import pandas as pd

In [4]:
plaza = pd.read_csv('people.csv', delimiter=";")

In [8]:
plaza.loc[plaza.Prof=='Guardia','Num'].sum()

44

### Fase 2: La Biblioteca 
Te encuentras en la biblioteca. La cantidad de libros es inmensa. Por suerte, en la plaza conseguiste lo que necesitas. Encuentra el libro que esté en la posición del número que has guardado y accede a su contenido.   
Descarga el archivo books.xml creando una conexión al servidor FTP  
`Servidor: f25-preview.runhosting.com   
Usuario: 3185129_EOI   
Contraseña: FTP_EOI1920 `

In [9]:
import ftplib
ftp = ftplib.FTP('f25-preview.runhosting.com','3185129_EOI','FTP_EOI1920')
filename = 'books.xml'
ftp.retrbinary("RETR "+filename,open(filename,'wb').write)

'226 Transfer complete'

In [10]:
ftp.quit()

'221 Goodbye.'

In [11]:
import xml.etree.ElementTree as ET
tree = ET.parse('books.xml')

In [21]:
raiz = tree.getroot()
print(raiz.tag)
lst = tree.findall('book')
print(lst[43].find('link').text)

root
http://johnstoniatexts.x10host.com/homer/iliad1html.html


### Fase 3: El Libro 
¡Rápido! El tiempo se agota. Para pasar a la siguiente frase deberás encontrar la palabra de seis letras que más se repite en el libro. ¡Usa tu magia!

In [33]:
import urllib.request
from bs4 import BeautifulSoup

url = 'http://johnstoniatexts.x10host.com/homer/iliad1html.html'
html = urllib.request.urlopen(url)
soup = BeautifulSoup(html)

tags = soup.find_all('span', class_="auto-style21")
texto = ''

for tag in tags:
    try:
        texto += tag.contents[0]
    except:
        None

contadores = dict() # Creo un diccionario para almacenar las palabras y contar las veces que aparece
texto = texto.replace(',','').replace('.','').replace(':','').replace('?','').replace('(','').replace(')','').replace('','') # Elimino signos de (puntos, comas, etc.)
palabras = texto.lower().split() # Paso todas las palabras a minúsculas (lower) y las corto palabra a palabra (split)

for palabra in palabras:
    if len(palabra)==6:
        contadores[palabra] = contadores.get(palabra,0) +1

df = pd.DataFrame(list(contadores.items()), columns=['palabra', 'contador']).sort_values('contador', ascending=False )
df

,palabra,contador
1,apollo,14
6,sacred,9
75,should,8
39,before,8
55,that’s,7
...,...,...
132,notson,1
133,ripped,1
134,bronze,1
137,namean,1


### Fase 4: La Tienda   
Tu siguiente destino es la tienda de comestibles cercana a la Iglesia. Solo te dejarán entrar si conoces la contraseña, la cual conseguiste en la fase anterior. Una vez dentro, averigua cuál es el producto más vendido en la historia de la tienda, y cómpralo. 
Descarga el archivo del siguiente enlace: https://cutt.ly/peVALXr, descomprímelo utilizando la respuesta de la fase anterior como contraseña. Carga la base de datos en Python, y encuentra el nombre 
del producto más vendido.


In [34]:
import sqlite3

In [35]:
conn = sqlite3.connect('tienda.sqlite')

In [36]:
pd.read_sql('SELECT name FROM sqlite_master WHERE type="table"',conn)

,name
0,Employee
1,Category
2,Customer
3,Shipper
4,Supplier
5,Order
6,Product
7,OrderDetail
8,CustomerCustomerDemo
9,CustomerDemographic


In [37]:
pd.read_sql('select * from OrderDetail',conn)

,Id,OrderId,ProductId,UnitPrice,Quantity,Discount
0,10248/11,10248,11,14.00,12,0.00
1,10248/42,10248,42,9.80,10,0.00
2,10248/72,10248,72,34.80,5,0.00
3,10249/14,10249,14,18.60,9,0.00
4,10249/51,10249,51,42.40,40,0.00
...,...,...,...,...,...,...
2150,11077/64,11077,64,33.25,2,0.03
2151,11077/66,11077,66,17.00,1,0.00
2152,11077/73,11077,73,15.00,2,0.01
2153,11077/75,11077,75,7.75,4,0.00


In [38]:
pd.read_sql('select * from Product',conn)

,Id,ProductName,SupplierId,CategoryId,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
0,1,Chai,1,1,10 boxes x 20 bags,18.00,39,0,10,0
1,2,Chang,1,1,24 - 12 oz bottles,19.00,17,40,25,0
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.00,13,70,25,0
3,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22.00,53,0,0,0
4,5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
72,73,Röd Kaviar,17,8,24 - 150 g jars,15.00,101,0,5,0
73,74,Longlife Tofu,4,7,5 kg pkg.,10.00,4,20,5,0
74,75,Rhönbräu Klosterbier,12,1,24 - 0.5 l bottles,7.75,125,0,25,0
75,76,Lakkalikööri,23,1,500 ml,18.00,57,0,20,0


In [41]:
query='''
SELECT SUM(Quantity) as Total,ProductName

FROM OrderDetail det
JOIN Product pro
ON det.ProductId = pro.Id
GROUP BY ProductName
ORDER BY Total DESC
'''
pd.read_sql(query,conn)

,Total,ProductName
0,1577,Camembert Pierrot
1,1496,Raclette Courdavault
2,1397,Gorgonzola Telino
3,1263,Gnocchi di nonna Alice
4,1158,Pavlova
...,...,...
72,184,Laughing Lumberjack Lager
73,138,Chocolade
74,125,Gravad lax
75,122,Genen Shouyu


### FASE 5: El Castillo   
¡Ya casi estás! Solo te falta entrar al castillo. Para ello, deberás sobornar al guardia elaborando la receta más calórica con el alimento que conseguiste en la fase anterior.

In [42]:
import urllib.request
import json

In [46]:
api_id = '0ceaaf83'
api_key = '93ff26f92c228dbafa3d7d2f9b789a39'

url = 'https://api.edamam.com/search'
q = 'Camembert'

url2 = url+'?q='+q+'&app_id='+api_id + '&app_key='+api_key

print(url2)

https://api.edamam.com/search?q=Camembert&app_id=0ceaaf83&app_key=93ff26f92c228dbafa3d7d2f9b789a39


In [47]:
datos = urllib.request.urlopen(url2).read()
js = json.loads(datos)
js

{'q': 'Camembert',
 'from': 0,
 'to': 10,
 'more': True,
 'count': 327,
 'hits': [{'recipe': {'uri': 'http://www.edamam.com/ontologies/edamam.owl#recipe_105bff17d2c146a02f784f68cfa0acb2',
    'label': 'Pan-fried camembert sandwich',
    'image': 'https://www.edamam.com/web-img/559/559bdee96acb7a8515065114ad075749.jpg',
    'source': 'BBC Good Food',
    'url': 'https://www.bbcgoodfood.com/recipes/pan-fried-camembert-sandwich',
    'shareAs': 'http://www.edamam.com/recipe/pan-fried-camembert-sandwich-105bff17d2c146a02f784f68cfa0acb2/camembert',
    'yield': 1.0,
    'dietLabels': ['Low-Carb'],
    'healthLabels': ['Sugar-Conscious',
     'Vegetarian',
     'Pescatarian',
     'Egg-Free',
     'Peanut-Free',
     'Tree-Nut-Free',
     'Soy-Free',
     'Fish-Free',
     'Shellfish-Free',
     'Pork-Free',
     'Red-Meat-Free',
     'Crustacean-Free',
     'Celery-Free',
     'Mustard-Free',
     'Sesame-Free',
     'Lupine-Free',
     'Mollusk-Free',
     'Alcohol-Free',
     'Sulfite-Fre

In [52]:
recetas = []
calorias = []

for receta in js['hits']:
    recetas.append(receta['recipe']['label'])
    calorias.append(receta['recipe']['calories'])

In [56]:
pd.DataFrame({'recetas':recetas,'calorias':calorias}).sort_values('calorias',ascending=False)

,recetas,calorias
2,Baked camembert,5911.433894
4,Puff pastry tarts with broccoli and Camembert ...,3147.970693
9,Camembert Crostini,1882.240209
8,Baked Camembert with Thyme & Garlic,1672.991124
7,Ham and Camembert Baguette Sandwich recipes,1416.203587
3,Baked Camembert with Fresh Fruit,1250.515222
5,Camembert Fondue Recipe,1027.489117
1,Baked Camembert with garlic bread,914.115000
6,Candied Camembert,632.250000
0,Pan-fried camembert sandwich,568.392857
